In [4]:
import pandas as pd

data = {
    "FeedbackID": [1,2,3,4,5,6,7,8,9,10],
    "CustomerID": [101,102,103,104,105,106,107,108,109,110],
    "Feedback": [
        "Great product, very satisfied!",
        "Delivery was late, disappointed.",
        "Excellent customer support.",
        "Product quality not as expected.",
        "Fast shipping and good packaging.",
        "The item arrived damaged.",
        "Amazing experience!",
        "Not worth the price.",
        "Satisfied with my purchase.",
        "Will recommend to friends."
    ],
    "Rating": [5,2,5,3,4,1,5,2,4,5],
    "Date": [
        "2025-01-05","2025-01-06","2025-01-07","2025-01-08","2025-01-09",
        "2025-01-10","2025-01-11","2025-01-12","2025-01-13","2025-01-14"
    ]
}

df = pd.DataFrame(data)

# Save CSV
df.to_csv("feedback.csv", index=False)

# Save Excel
df.to_excel("feedback.xlsx", sheet_name="FeedbackSheet", index=False)

# Save JSON
df.to_json("feedback.json", orient="records", indent=4)

print("✅ feedback.csv, feedback.xlsx, and feedback.json created")



✅ feedback.csv, feedback.xlsx, and feedback.json created


In [5]:
import pandas as pd
import sqlite3

# 1️Read CSV, Excel, JSON

df_csv = pd.read_csv("feedback.csv")
df_excel = pd.read_excel("feedback.xlsx", sheet_name="FeedbackSheet")
df_json = pd.read_json("feedback.json")


# 2️ Convert textual feedback into structured DataFrame

# Already structured with columns: FeedbackID, CustomerID, Feedback, Rating, Date
# Convert Date column to datetime
df_csv["Date"] = pd.to_datetime(df_csv["Date"])
df_excel["Date"] = pd.to_datetime(df_excel["Date"])
df_json["Date"] = pd.to_datetime(df_json["Date"])

# 3️ Merge all sources

df_merged = pd.concat([df_csv, df_excel, df_json], ignore_index=True)

# 4️Handle missing values

# Check missing
print("Missing values:\n", df_merged.isnull().sum())

# Fill missing Ratings with average rating
df_merged["Rating"] = df_merged["Rating"].fillna(df_merged["Rating"].mean())

# Fill missing Feedback with placeholder
df_merged["Feedback"] = df_merged["Feedback"].fillna("No feedback provided")

# 5️ Remove duplicates

df_merged = df_merged.drop_duplicates(subset=["FeedbackID"])
print("\nUnique feedback records:", len(df_merged))

# 6️ Store in SQLite

conn = sqlite3.connect("feedback.db")
df_merged.to_sql("customer_feedback", conn, if_exists="replace", index=False)
df_sql = pd.read_sql("SELECT * FROM customer_feedback", conn)
print("\nSQLite Data Preview:\n", df_sql.head())

# 7️ Export cleaned feedback to CSV, Excel, JSON

df_merged.to_csv("cleaned_feedback.csv", index=False)
df_merged.to_excel("cleaned_feedback.xlsx", sheet_name="All_Feedback", index=False)
df_merged.to_json("cleaned_feedback.json", orient="records", indent=4)

print("✅ Cleaned feedback exported to CSV, Excel, and JSON")


Missing values:
 FeedbackID    0
CustomerID    0
Feedback      0
Rating        0
Date          0
dtype: int64

Unique feedback records: 10

SQLite Data Preview:
    FeedbackID  CustomerID                           Feedback  Rating  \
0           1         101     Great product, very satisfied!       5   
1           2         102   Delivery was late, disappointed.       2   
2           3         103        Excellent customer support.       5   
3           4         104   Product quality not as expected.       3   
4           5         105  Fast shipping and good packaging.       4   

                  Date  
0  2025-01-05 00:00:00  
1  2025-01-06 00:00:00  
2  2025-01-07 00:00:00  
3  2025-01-08 00:00:00  
4  2025-01-09 00:00:00  
✅ Cleaned feedback exported to CSV, Excel, and JSON
